Linking Folders and Files + Installing dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/GaParmar/img2img-turbo.git
%cd img2img-turbo
!pip install -q -U pip
!pip install -q -r requirements.txt

# if T4
!pip install -q xformers accelerate==0.34.2

#checking all g with working dir
!pwd && ls

In [ ]:
# Pointing to dataset loc
import os, pathlib, shutil

drive_data_root = "/content/drive/MyDrive/cycleGAN_dataset_test"  # folder matching required training folder structure (train_A, train_B, test_A, test_B + fixed prompts)
dataset_name = "my_sat2uav"

repo_root = "/content/img2img-turbo"
data_dst = f"{repo_root}/data/{dataset_name}"
os.makedirs(f"{repo_root}/data", exist_ok=True)

# Since dataset is already organised as specified, symlink easier to implement.
if os.path.exists(data_dst):
    shutil.rmtree(data_dst)
os.symlink(drive_data_root, data_dst, target_is_directory=True)

# Double checking file listing
for split in ("train_A","train_B","test_A","test_B"):
    print(split, "=>", len(list(pathlib.Path(data_dst, split).glob("*"))), "files")


train_A => 50 files
train_B => 29 files
test_A => 5 files
test_B => 3 files


In [ ]:
# Creating/modifying the required fixed prompts dataset (as per img2img training doc).
open(f"{data_dst}/fixed_prompt_a.txt","w").write("overhead satellite scene")
open(f"{data_dst}/fixed_prompt_b.txt","w").write("overhead uav scene")


18

In [ ]:
!pwd && ls


/content/img2img-turbo
assets	environment.yaml	LICENSE		  scripts
data	gradio_canny2image.py	README.md	  src
docs	gradio_sketch2image.py	requirements.txt  style.css


Initilising Environment

In [ ]:
# Configures accelerate with safe defaults
from accelerate.utils import write_basic_config
write_basic_config(mixed_precision="fp16")
print("Wrote Accelerate default config.")

# if above modification of defaults not working, standard defaults useable
# accelerate config

Wrote Accelerate default config.


Training Model

In [ ]:
## Training code including script/flags pattern from the official doc (for reference if my code modified too much).

# export NCCL_P2P_DISABLE=1
# accelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \
#     --pretrained_model_name_or_path="stabilityai/sd-turbo" \
#     --output_dir="output/cyclegan_turbo/my_horse2zebra" \
#     --dataset_folder "data/my_horse2zebra" \
#     --train_img_prep "resize_286_randomcrop_256x256_hflip" --val_img_prep "no_resize" \
#     --learning_rate="1e-5" --max_train_steps=25000 \
#     --train_batch_size=1 --gradient_accumulation_steps=1 \
#     --report_to "wandb" --tracker_project_name "gparmar_unpaired_h2z_cycle_debug_v2" \
#     --enable_xformers_memory_efficient_attention --validation_steps 250 \
#     --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1

In [ ]:
# Training code — simpler settings for small, unpaired dataset on T4 (5000 steps instead of 25000).
%%bash
cd /content/img2img-turbo
export NCCL_P2P_DISABLE=1

# export TF_CPP_MIN_LOG_LEVEL=3 #minimises TF logs
accelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \
  --pretrained_model_name_or_path="stabilityai/sd-turbo" \
  --output_dir="output/cyclegan_turbo/my_sat2uav" \
  --dataset_folder="data/my_sat2uav" \
  --train_img_prep "resize_512_hflip" --val_img_prep "no_resize" \
  --learning_rate="1e-5" \
  --max_train_steps=5000 \
  --train_batch_size=1 --gradient_accumulation_steps=1 \
  --report_to "tensorboard" \
  --tracker_project_name "sat2uav-local" \
  --enable_xformers_memory_efficient_attention --validation_steps 250 \
  --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1



2025-10-28 04:49:29.181423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761626969.201203    8373 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761626969.207159    8373 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761626969.222190    8373 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761626969.222212    8373 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761626969.222214    8373 computation_placer.cc:177] computation placer alr

CalledProcessError: Command 'b'cd /content/img2img-turbo\nexport NCCL_P2P_DISABLE=1\n# export TF_CPP_MIN_LOG_LEVEL=3 #minimises TF logs\naccelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \\\n  --pretrained_model_name_or_path="stabilityai/sd-turbo" \\\n  --output_dir="output/cyclegan_turbo/my_sat2uav" \\\n  --dataset_folder="data/my_sat2uav" \\\n  --train_img_prep "resize_512_hflip" --val_img_prep "no_resize" \\\n  --learning_rate="1e-5" \\\n  --max_train_steps=5000 \\\n  --train_batch_size=1 --gradient_accumulation_steps=1 \\\n  --report_to "tensorboard" \\\n  --tracker_project_name "sat2uav-local" \\\n  --enable_xformers_memory_efficient_attention --validation_steps 250 \\\n  --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1\n\n'' returned non-zero exit status 1.

In [ ]:
%%bash
cd /content/img2img-turbo
export NCCL_P2P_DISABLE=1
export TF_CPP_MIN_LOG_LEVEL=3
pip -q install wandb >/dev/null 2>&1


export WANDB_MODE=offline
export WANDB_PROJECT=sat2uav-local
export WANDB_DISABLE_CODE=true

accelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \
  --pretrained_model_name_or_path="stabilityai/sd-turbo" \
  --output_dir="output/cyclegan_turbo/my_sat2uav" \
  --dataset_folder="data/my_sat2uav" \
  --train_img_prep "resize_512_hflip" --val_img_prep "no_resize" \
  --learning_rate="1e-5" \
  --max_train_steps=5000 \
  --train_batch_size=1 --gradient_accumulation_steps=1 \
  --report_to "wandb" \
  --tracker_project_name "sat2uav-local" \
  --enable_xformers_memory_efficient_attention \
  --validation_steps 250 \
  --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1


E0000 00:00:1761627246.845642    9553 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761627246.857015    9553 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761627246.884348    9553 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761627246.884425    9553 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761627246.884442    9553 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761627246.884456    9553 computation_placer.cc:177] computation placer already registered. Please check linka

CalledProcessError: Command 'b'cd /content/img2img-turbo\nexport NCCL_P2P_DISABLE=1\nexport TF_CPP_MIN_LOG_LEVEL=3           # optional: quiet TensorFlow\npip -q install wandb >/dev/null 2>&1    # if not already installed\n\n# Make W&B completely offline\nexport WANDB_MODE=offline\nexport WANDB_PROJECT=sat2uav-local      # matches your --tracker_project_name\nexport WANDB_DISABLE_CODE=true\n\naccelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \\\n  --pretrained_model_name_or_path="stabilityai/sd-turbo" \\\n  --output_dir="output/cyclegan_turbo/my_sat2uav" \\\n  --dataset_folder="data/my_sat2uav" \\\n  --train_img_prep "resize_512_hflip" --val_img_prep "no_resize" \\\n  --learning_rate="1e-5" \\\n  --max_train_steps=5000 \\\n  --train_batch_size=1 --gradient_accumulation_steps=1 \\\n  --report_to "wandb" \\\n  --tracker_project_name "sat2uav-local" \\\n  --enable_xformers_memory_efficient_attention \\\n  --validation_steps 250 \\\n  --lambda_gan 0.5 --lambda_idt 1 -

Inference Test

In [ ]:
# Quick inference test (A->B) on a sample from your test_A
%%bash
python src/inference_unpaired.py \
  --model_path "output/cyclegan_turbo/my_sat2uav/checkpoints/model_1001.pkl" \
  --input_image "data/my_sat2uav/test_A/$(ls data/my_sat2uav/test_A | head -n1)" \
  --prompt "overhead uav scene" --direction "a2b" \
  --output_dir "outputs" --image_prep "no_resize"


Visualising

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob, os

direction = "a2b"  # sat2uav or "b2a" (uav2sat)
repo_root = "/content/img2img-turbo"
input_dir = os.path.join(repo_root, f"data/my_sat2uav/test_{'A' if direction=='a2b' else 'B'}")
output_dir = os.path.join(repo_root, "outputs")

out_files = sorted(glob.glob(os.path.join(output_dir, "*.png")) + glob.glob(os.path.join(output_dir, "*.jpg")))

if not out_files:
    print("No output images found.")
else:
    print(f"Found {len(out_files)} output images:")
    for output_path in out_files:
        base_name = os.path.splitext(os.path.basename(output_path))[0]
        cand_inputs = glob.glob(os.path.join(input_dir, f"{base_name}.*"))
        input_path = cand_inputs[0] if cand_inputs else None

        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        if input_path:
            axs[0].imshow(mpimg.imread(input_path))
            axs[0].set_title(f"Input ({'Satellite' if direction=='a2b' else 'UAV'})")
        else:
            axs[0].text(0.5,0.5,"No matching input",ha="center",va="center")
        axs[0].axis("off")

        axs[1].imshow(mpimg.imread(output_path))
        axs[1].set_title(f"Generated ({'UAV-style' if direction=='a2b' else 'Satellite-style'})")
        axs[1].axis("off")

        plt.tight_layout()
        plt.show()




Saving Externally

In [ ]:
!mkdir -p /content/drive/MyDrive/cyclegan_outputs
!cp /content/img2img-turbo/outputs/* /content/drive/MyDrive/cyclegan_outputs/
print("Outputs now in Drive folder: /content/drive/MyDrive/cyclegan_outputs/")
